In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Conv2D, Input, MaxPool2D

In [ ]:
SEGMENTATED_DATA_FOLDER = '../../data/PASCAL/VOC2012/SegmentationObject'
ORIGINAL_DATA_FOLDER = '../../data/PASCAL/VOC2012/JPEGImages'

In [ ]:
img = cv.imread(f"{ORIGINAL_DATA_FOLDER}/2007_000999.jpg")
img2 = cv.imread(f"{ORIGINAL_DATA_FOLDER}/2007_000925.jpg")

img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
img2 = cv.cvtColor(img2, cv.COLOR_BGR2RGB)

mask = cv.imread(f"{SEGMENTATED_DATA_FOLDER}/2007_000999.png", cv.IMREAD_GRAYSCALE)
mask2 = cv.imread(f"{SEGMENTATED_DATA_FOLDER}/2007_000925.png", cv.IMREAD_GRAYSCALE)

print(f"Image shape: {mask.shape}")
print(f"Image 2 shape: {mask2.shape}")

plt.imshow(img)
plt.show()
plt.imshow(img2)
plt.show()


In [ ]:
def create_std_unet():

    contracting = contracting_path()
    expanding = expanding_path(contracting)

def contracting_path():
    inputs = Input(shape=(572, 572, 1))                      # 572x572x1

    conv_1_1 = Conv2D(64, 3, activation='relu')(inputs)      # 570x570x64
    conv_1_2 = Conv2D(64, 3, activation='relu')(conv_1_1)    # 568x568x64

    pool_1 = MaxPool2D((2,2), strides=2)(conv_1_2)           # 284x284x64

    conv_2_1 = Conv2D(128, 3, activation='relu')(pool_1)     # 282x282x128
    conv_2_2 = Conv2D(128, 3, activation='relu')(conv_2_1)   # 280x280x128

    pool_2 = MaxPool2D((2,2,), strides=2)(conv_2_2)          # 140x140x128

    conv_3_1 = Conv2D(256, 3, activation='relu')(pool_2)     # 138x138x256
    conv_3_2 = Conv2D(256, 3, activation='relu')(conv_3_1)   # 136x136x256

    pool_3 = MaxPool2D((2,2), strides=2)(conv_3_2)           # 68x68x256

    conv_4_1 = Conv2D(512, 3, activation='relu')(pool_3)     # 66x66x512
    conv_4_2 = Conv2D(512, 3, activation='relu')(conv_4_1)   # 64x64x512

    pool_4 = MaxPool2D((2,2), strides=2)(conv_4_2)           # 32x32x512

    conv_5_1 = Conv2D(1024, 3, activation='relu')(pool_4)    # 30x30x1024
    conv_5_2 = Conv2D(1024, 3, activation='relu')(conv_5_1)  # 28x28x1024

    return conv_5_2

def expanding_path():
    pass